In [1]:
import numpy as np
import cvxpy as cp


###  Lab 09
### K. Surya Prakash
### ee18btech11026

# Question 01

## Code

In [3]:
## Data : 

#h,w,d = cp.Variable(),cp.Variable(),cp.Variable()

#y1,y2,y3 = cp.log(h),cp.log(w),cp.log(d)
y1,y2,y3 = cp.Variable(),cp.Variable(),cp.Variable()

## Objective : 
obj = cp.Minimize( -1*(y1+y2+y3) )


constraints = [cp.log( cp.exp(y1+y2) + cp.exp(y1+y3) )<= cp.log(100),
              cp.log( cp.exp(y2+y3) ) <= cp.log(60),
               
              cp.log( cp.exp(y1-y2) ) <= cp.log(1.2),
              cp.log( cp.exp(y3-y2) ) <= cp.log(1.2),
              -1*cp.log( cp.exp(y1-y2) ) <= -1*cp.log(0.8),
              -1*cp.log( cp.exp(y1-y2) ) <= -1*cp.log(0.8),
              ]

prob = cp.Problem(obj, constraints)
prob.solve(qcp = True)




h = np.exp(y1.value)
w = np.exp(y2.value)
d = np.exp(y3.value)




/home/surya/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/home/surya/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/home/surya/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1061: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


## Results 

In [4]:
print('The optimum dimensions are')
print('h = ',round(h,3))
print('w = ',round(w,3) )
print('d = ', round(d,3))

print("\nOptimum volume : ", round(h*w*d ,3))

The optimum dimensions are
h =  6.455
w =  7.746
d =  7.746

Optimum volume :  387.301


#  Question : 02


In [16]:
### Data  : 

sigma = np.ones((5,1))
G = np.array([[1,0.1,0.2,0.1,0],
             [0.1,1,0.1,0.1,0],
            [0.2,0.1,2,0.2,0.2],
              [0.1,0.1,0.2,1,0.1],
              [0,0,0.2,0.1,1]])

print("G \n--------")
print(G)


### Creating Gs and Gi
Gs = np.zeros(G.shape)
for i in range(G.shape[0]):
    Gs[i][i] = G[i][i]
    
Gi = G- Gs



G 
--------
[[1.  0.1 0.2 0.1 0. ]
 [0.1 1.  0.1 0.1 0. ]
 [0.2 0.1 2.  0.2 0.2]
 [0.1 0.1 0.2 1.  0.1]
 [0.  0.  0.2 0.1 1. ]]


## Formulation

## Code

In [43]:
#####   Hyperparameters setting

# choosing upper and lower bounds for sub-level set
u = 10
l = 0.0
eps = .0001  ### Epsilon

p = cp.Variable((5,1))
s = Gs@p  ## signal power @ Rx
i = Gi@p  ## interference power @Rx

i = i + 0.5  ##  Adding inherent noise std (0.5)


grp_1 = p[0] + p[1]   ## Total power of grp1
grp_2 = p[2] + p[3] + p[4]  ## Total power of grp2


## defining t : check all the sublevel sets below this 
### Remember min:SINR (1/(best t))

t = cp.Parameter(shape = 1)
obj = cp.Minimize(t)

constraints = [i<= t*s ,p<=3,p>=0, G@p <= 5,grp_1<=4, grp_2<=6 ]


## Sanity check for the range :
t.value = [u]
prob = cp.Problem(obj, constraints)
prob.solve()

##### Check if we can obtain a optimal value for u :: if not increase u
if (prob.status != 'optimal'):
    print(" Solution does not exist in [l,u] -- > increase u")



while (u-l >= eps):      ### iterating until the range is small
    t.value = np.atleast_1d((u + l)/2.0)
    prob = cp.Problem(obj,constraints)
    prob.solve()
    
    if (prob.status == 'optimal'):
        u = t.value
        p_opt = p.value
        t_opt = t.value
    else: 
        l = t.value
    

print('The optimum value is achieved at t = ',(t.value))

The optimum value is achieved at t =  [0.5922699]


## Results

In [40]:
### Results : 

i_opt = Gi@p_opt + 0.5
s_opt = Gs@p_opt

sinr_opt = s_opt/i_opt

print("The Tx Powers are : ",p_opt[:,0])
print('\n')
print('The Rx SINR are : ', sinr_opt[:,0])
print('\n')
print("Max possible value of minimum of all sinr is ",min(sinr_opt))

The Tx Powers are :  [2.11867997 1.88110908 1.64463325 2.37923023 1.80295966]


The Rx SINR are :  [1.68824424 1.68822234 1.68828826 1.68835343 1.68998474]


Max possible value of minimum of all sinr is  [1.68822234]
